# Assignment

-------------

**1) Complete this notebook and make a pull request:** 

Answer questions (Q) in the space provided (A) in this notebook. When finished, copy your notebook to the `Assignment/` directory and name it `nb-6.5-<Github-username>.ipynb`. Then make a pull request to the upstream repo. The entered answers in this notebook will be simply Markdown text where I want you to interpret and describe a block of code to better understand what it is doing. Much of this code you will have seen already. 


**2) Write an importable Python package, save as a repo, and test it here.**

The package should be written as we did in our last lession (`.py` files in a directory with a setup.py file so it can be installed with `pip`). Follow instructions at the end of this notebook for how to write your package. Test it here by importing the package and executing the code at the end. It should work and give correct answers, if not, continue working on it. When you have it completed save your package as a new Github repo named `seqlib`.

### The `seqlib` package

Together we are going to write several functions here that will make up your new package called `seqlib`. It will be your job to copy these functions, organize them into a Class, save the code into a `.py` file (you can use SublimeText if you're comfortable with it for much of this, or any text editor including the one in jupyter), package the files so they can be imported as a library, and test the package so that it accomplishes the tasks which are defined at the end of this notebook. First things first, though, let's write the functions. 

In [1]:
import numpy as np
import pandas as pd

### Q.  Describe what the `mutate()` function below does:


A. `mutate()` selects a random nucleotide pair other than the input one.

In [2]:
def mutate(base):
    diff = set("ACTG") - set(base)
    return np.random.choice(list(diff))

In [3]:
# test it
mutate("A")

'C'

### Q. Describe how the `simulate()` function below works:
Annotate the code by inserting lines with comments as you read through the function to make sense of it. What is being created at each step and how is it used?


A. `simulate()` returns nind variable DNA sequences of length nsites (with random missing data).

In [5]:
def simulate(ninds, nsites):
    oseq = np.random.choice(list("ACGT"), size=nsites)         # produces a random DNA sequence of length nsites
    arr = np.array([oseq for i in range(ninds)])               # makes nind copies of the sequence
    muts = np.random.binomial(1, 0.1, (ninds, nsites))         # selects sites for mutation with probability 0.1
    for col in range(nsites):                                  # mutates sites 
        newbase = mutate(arr[0, col])
        mask = muts[:, col].astype(bool)
        arr[:, col][mask] = newbase
    missing = np.random.binomial(1, 0.1, (ninds, nsites))      # selects sites with missing data with probability 0.1
    arr[missing.astype(bool)] = "N"                            # replace the sites by 'N'
    return arr

In [6]:
seqs = simulate(6, 15)
print(seqs)

[['C' 'N' 'C' 'T' 'C' 'G' 'C' 'C' 'T' 'C' 'A' 'C' 'A' 'A' 'A']
 ['C' 'A' 'C' 'T' 'C' 'G' 'N' 'N' 'T' 'C' 'A' 'C' 'C' 'A' 'N']
 ['C' 'A' 'C' 'T' 'N' 'G' 'G' 'C' 'T' 'C' 'A' 'C' 'C' 'N' 'A']
 ['C' 'A' 'N' 'N' 'C' 'G' 'G' 'C' 'T' 'C' 'A' 'A' 'C' 'T' 'A']
 ['N' 'A' 'A' 'T' 'C' 'G' 'G' 'C' 'T' 'C' 'A' 'C' 'C' 'N' 'A']
 ['C' 'N' 'C' 'T' 'C' 'G' 'G' 'C' 'T' 'C' 'N' 'C' 'C' 'A' 'A']]


### **Q: Describe how the `filter_missing` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it find columns with missing (N) values in them? How might you mprove it?

A. Selects only the sites below a maximum frequency of missing data.

In [7]:
def filter_missing(arr, maxfreq):
    freqmissing = np.sum(arr == "N", axis=0) / arr.shape[0]       # calculates the frequency of Ns along the columns
    return arr[:, freqmissing <= maxfreq]                         # returns only sites where freqmissing is below maxfreq

In [8]:
filter_missing(seqs, 0.1)

array([['G', 'T', 'C', 'C', 'A'],
       ['G', 'T', 'C', 'C', 'C'],
       ['G', 'T', 'C', 'C', 'C'],
       ['G', 'T', 'C', 'A', 'C'],
       ['G', 'T', 'C', 'C', 'C'],
       ['G', 'T', 'C', 'C', 'C']], dtype='<U1')

### **Q: Describe how the `filter_maf` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it calculate minor allele frequencies? Why does it use copy?

A. `filter_maf` returns only sites with SNPs above a minimum frequency.

In [9]:
def filter_maf(arr, minfreq):
    freqs = np.sum(arr != arr[0], axis=0) / arr.shape[0]     # calculates the frequency of the first nucleotide in each site
    maf = freqs.copy()                                       # copies the frequencies
    maf[maf > 0.5] = 1 - maf[maf > 0.5]                      # for frequencies above 0.5, replaces by the minor freq instead
    return arr[:, maf > minfreq]                             # returns the filtered data

In [10]:
filter_maf(seqs, 0.1)

array([['C', 'N', 'C', 'T', 'C', 'C', 'C', 'A', 'C', 'A', 'A', 'A'],
       ['C', 'A', 'C', 'T', 'C', 'N', 'N', 'A', 'C', 'C', 'A', 'N'],
       ['C', 'A', 'C', 'T', 'N', 'G', 'C', 'A', 'C', 'C', 'N', 'A'],
       ['C', 'A', 'N', 'N', 'C', 'G', 'C', 'A', 'A', 'C', 'T', 'A'],
       ['N', 'A', 'A', 'T', 'C', 'G', 'C', 'A', 'C', 'C', 'N', 'A'],
       ['C', 'N', 'C', 'T', 'C', 'G', 'C', 'N', 'C', 'C', 'A', 'A']],
      dtype='<U1')

### Q: What order should these functions be applied, does it matter?

A. The other does matter. Missing data should be removed first, otherwise it is counted as variable site.

In [11]:
filter_missing(filter_maf(seqs, 0.1), 0.1)

array([['C', 'A'],
       ['C', 'C'],
       ['C', 'C'],
       ['A', 'C'],
       ['C', 'C'],
       ['C', 'C']], dtype='<U1')

In [12]:
filter_maf(filter_missing(seqs, 0.1), 0.1)

array([['C', 'A'],
       ['C', 'C'],
       ['C', 'C'],
       ['A', 'C'],
       ['C', 'C'],
       ['C', 'C']], dtype='<U1')

### Q: Describe how `calculate_statistics()` works


A. `calculate_statistics()` calculates mean nucleotide diversity, mean minor allele frequency, and the number of invariant and variable sites.

In [13]:
def calculcate_statistics(arr):
    nd = np.var(arr == arr[0], axis=0).mean()
    mf = np.mean(np.sum(arr != arr[0], axis=0) / arr.shape[0])
    inv = np.any(arr != arr[0], axis=0).sum()
    var = arr.shape[1] - inv
    return pd.Series(
        {"mean nucleotide diversity": nd,
         "mean minor allele frequency": mf,
         "invariant sites": inv,
         "variable sites": var,
        })

In [14]:
calculcate_statistics(seqs)

invariant sites                12.000000
mean minor allele frequency     0.288889
mean nucleotide diversity       0.129630
variable sites                  3.000000
dtype: float64

### Instructions: Write a `seqlib` Class object

I started writing the bare bones of it below. You should write it so that it can be executed as described below to perform all of the functions we defined above, and so that its attributes can be accessed. Save this class object in a `.py` file and make it into an importable package called `seqlib`. You can write and test your object in this notebook if you like, but it must be saved separately in a `.py` file and be imported. You cannot execute the code at the end using your object defined here in the notebook. When finished save your package to GitHub as a repo just like we did with the `helloworld` package. You do not need to write a CLI script like we did for the `helloworld` package, we will only be using the Python API here. See the examples below for **how you should write your Class object**. It should be able to run in the way written below, so look at that code and think about how you would write a Class object that can do that. 

While you can mostly copy the functions from above, you will need to modify them slightly to access information about the Class object using *self*. For example, the `simulate()` function below takes self as a first argument and can access `self.inds` and `self.nsites` from that, so we do not need to provide those as arguments to the `simulate` function like we did above. 

In [ ]:
class Seqlib:
    def __init__(self, ninds, nsites):
        self.ninds = ninds
        self.nsites = nsites
        self.seqs = self.simulate()
        # ...
        
    def simulate(self):
        pass
        # ...
        
    # continue writing this full object...

## Test your package
The package should be globally importable (you ran `pip install .` or `pip install -e .` to install it), and it should be able to execute the following code without error. 

In [50]:
import seqlib

In [51]:
# init a Seqlib Class object
seqs = seqlib.Seqlib(ninds=10, nsites=50)

NameError: name 'mutate' is not defined

In [49]:
# access attributes from the object
print(seqs.ninds, seqs.nsites)

AttributeError: 'numpy.ndarray' object has no attribute 'ninds'

In [ ]:
# returns the MAF of the array as an array of floats
seqs.maf

In [ ]:
# return a view of the filtered sequence array by applying a new function 
# called `filter()` that applies both the maf and missing filter functions
seqs.filter(minmaf=0.1, maxmissing=0.0)

In [ ]:
# calculate statistics for an array with the results returned as a DataFrame
seqs.calculate_statistics()

In [ ]:
# calculate statistics for an array after filtering it
seqs.filter(minmaf=0.1, maxmissing=0.0).calculate_statistics()